In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data
import urllib.request
import gzip
import shutil
import os

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolution layer: 1 input channel (grayscale), 16 output channels, 3x3 filter
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        # Pooling layer: reduces spatial dimensions by half
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layer: input is flattened feature maps, output is number of classes (e.g., 10)
        self.fc1 = nn.Linear(16 * 14 * 14, 10)  # assuming input images are 28x28
        
    def forward(self, x):
        # Convolution -> ReLU Activation -> Pooling
        x = self.pool(F.relu(self.conv1(x)))
        # Flatten the feature maps
        x = x.view(x.size(0), -1)
        # Fully connected layer for final output
        x = self.fc1(x)
        return x


In [15]:
# Define the transformation
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize to 28x28 pixels
    transforms.ToTensor(),        # Convert to tensor
])

# Function to download and extract MNIST dataset manually
def download_mnist(url, filename):
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, filename)
        print(f"Extracting {filename}...")
        with gzip.open(filename, 'rb') as f_in:
            with open(filename.replace('.gz', ''), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"{filename} downloaded and extracted.")

# URLs for MNIST dataset
urls = [
    ("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "./data/train-images-idx3-ubyte.gz"),
    ("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz", "./data/train-labels-idx1-ubyte.gz"),
    ("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "./data/t10k-images-idx3-ubyte.gz"),
    ("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz", "./data/t10k-labels-idx1-ubyte.gz")
]

# Download and extract the dataset
for url, filename in urls:
    download_mnist(url, filename)

# Load the dataset
dataset = datasets.MNIST(root='./data', train=True, download=False, transform=transform)

# Create a DataLoader
dataloader = data.DataLoader(dataset, batch_size=32, shuffle=True)


HTTPError: HTTP Error 404: Not Found

In [ ]:
# Define the model (assuming SimpleCNN is already defined)
model = SimpleCNN()

# Iterate over the DataLoader
for images, labels in dataloader:
    # Pass the batch of images to the model
    output = model(images)
    print(output.shape)
    # You can add your training loop here